In [1]:
import sys
sys.path.insert(0, '../qdmt/')

In [2]:
from uMPSHelpers import *
from optimise import genAContr

In [3]:
import numpy as np
from scipy.stats import unitary_group

In [4]:
def uniformToRhoN(A, N, l=None, r=None):
    '''
    Generate $\rho(A)_N$ which is the density matrix over $N$ sites.
    '''
    if l is None or r is None:
        l, r = fixedPoints(A)
    l, r = fixedPoints(A)

    tensors = [l, *[A]*N, *[A.conj()]*N, r]
    edges = [(2, 1)]
    i = 3
    edgesA = [(1, -1, i)]
    edgesAdag = [(2, -N-1, i+1)]
    i += 2
    for j in range(N-1):
        edgesA.append((i-2, -2-j, i))
        edgesAdag.append((i-1, -N-2-j, i+1))
        i += 2

    i = edgesA[-1][2]
    j = edgesAdag[-1][2]
    edges = edges + edgesA + edgesAdag + [(i, j)]

    print(edges)
    return ncon(tensors, edges)


## Implement evolution 

In [5]:
# Initialise evolution unitary 
d = 2
D = 4
U = unitary_group.rvs(d**2)
U_tensor = U.reshape(d, d, d, d)

In [6]:
A = createMPS(D, d)
A = normalizeMPS(A)

In [7]:
N = 4
rhoA = uniformToRhoN(A, N=N)

[(2, 1), (1, -1, 3), (3, -2, 5), (5, -3, 7), (7, -4, 9), (2, -5, 4), (4, -6, 6), (6, -7, 8), (8, -8, 10), (9, 10)]


In [8]:
rhoM = rhoA.reshape(d**N, d**N)

In [9]:
UU = ncon([U, U], ((-1, -3), (-2, -4))).reshape(d**N, d**N)

In [10]:
rhoMUU = UU @ rhoM @ UU.conj().T
rhoMUU = rhoMUU.reshape(*[d]*(N*2))

In [11]:
def evolve_rho_first_layer(A, U, N, l=None, r=None):
    if l is None or r is None: 
        l, r = fixedPoints(A)
        
    assert N % 2 == 0, 'N needs to be even'
    Acontr = genAContr(1, 3, 5, 4, N)
    AdagContr = genAContr(2, 4, 6, 4, N)
    # print('Adagcontr: ', AdagContr)
    
    botLeg1 = [a[1] for a in Acontr[::2]]
    botLeg2 = [a[1] for a in Acontr[1::2]]
    topLeg1 = range(-1, -1*N-1, -2)
    topLeg2 = range(-2, -1*N-1, -2)
    
    Ucontr = [list(a) for a in zip(topLeg1, topLeg2, botLeg1, botLeg2)]
    # print(Udagcontr)
    
    botLeg1 = [a[1] for a in AdagContr[::2]]
    botLeg2 = [a[1] for a in AdagContr[1::2]]
    topLeg1 = range(-7, -N-8, -2)
    topLeg2 = range(-8, -N-9, -2)
    
    # print(topLeg1)
    # print(list(topLeg1))
    # print(topLeg2)
    # print(list(topLeg2))
    Udagcontr = [list(a) for a in zip(topLeg1, topLeg2, botLeg1, botLeg2)]
    # print(Ucontr)
    
    tensors = [l] + [A]*N + [A.conj()]*N + [U.conj()]*(N//2) + [U]*(N//2) + [r]
    contr = [[AdagContr[0][0], Acontr[0][0]]] + Acontr + AdagContr + Udagcontr + Ucontr + [[Acontr[-1][-1], AdagContr[-1][-1]]] 
    print(contr)
    
    return ncon(tensors, contr)

In [12]:
evolvedRhoTensor = evolve_rho_first_layer(A, U_tensor, 4)

[[2, 1], [1, 3, 5], [5, 7, 9], [9, 11, 13], [13, 15, 17], [2, 4, 6], [6, 8, 10], [10, 12, 14], [14, 16, 18], [-7, -8, 4, 8], [-9, -10, 12, 16], [-1, -2, 3, 7], [-3, -4, 11, 15], [17, 18]]


In [13]:
tensors = [U_tensor, U_tensor, U_tensor.conj(), U_tensor.conj(), rhoA]
contr = [
    (-1, -2, 1, 2), (-3, -4, 3, 4), (-5, -6, 5, 6), (-7, -8, 7, 8), 
    (1, 2, 3, 4, 5, 6, 7, 8)
]
evolvedRhoUtensor = ncon(tensors, contr)

In [14]:
np.allclose(evolvedRhoTensor, evolvedRhoUtensor)

True

In [15]:
np.allclose(rhoMUU, evolvedRhoTensor)

True

In [16]:
np.allclose(rhoMUU, evolvedRhoUtensor)

True

In [17]:
testrhoMUU = ncon([UU, rhoM, UU.conj()], ((-1, 1), (1, 2), (-2, 2)))

In [18]:
np.allclose(rhoMUU.reshape(d**N, d**N), testrhoMUU)

True

## Apply even and odd unitaries

In [19]:
# Initialise evolution unitary 
d = 2
D = 4
N = 4
U = unitary_group.rvs(d**2)
U_tensor = U.reshape(d, d, d, d)

In [20]:
A = createMPS(D, d)
A = normalizeMPS(A)

In [ ]:
def evolve_rho_first_layer(A, U, N, l=None, r=None):
    if l is None or r is None: 
        l, r = fixedPoints(A)
        
    assert N % 2 == 0, 'N needs to be even'
    Acontr = genAContr(1, 3, 5, 4, N)
    AdagContr = genAContr(2, 4, 6, 4, N)
    
    botLeg1 = [a[1] for a in Acontr[::2]]
    botLeg2 = [a[1] for a in Acontr[1::2]]
    topLeg1 = range(-1, -1*N-1, -2)
    topLeg2 = range(-2, -1*N-1, -2)
    
    Ucontr = [list(a) for a in zip(topLeg1, topLeg2, botLeg1, botLeg2)]
    
    botLeg1 = [a[1] for a in AdagContr[::2]]
    botLeg2 = [a[1] for a in AdagContr[1::2]]
    topLeg1 = range(-7, -N-8, -2)
    topLeg2 = range(-8, -N-9, -2)
    
    Udagcontr = [list(a) for a in zip(topLeg1, topLeg2, botLeg1, botLeg2)]
    
    tensors = [l] + [A]*N + [A.conj()]*N + [U.conj()]*(N//2) + [U]*(N//2) + [r]
    contr = [[AdagContr[0][0], Acontr[0][0]]] + Acontr + AdagContr + Udagcontr + Ucontr + [[Acontr[-1][-1], AdagContr[-1][-1]]] 
    
    print(contr)
    return ncon(tensors, contr)

In [71]:
def genUContr(topLeft, topRight, botLeft, botRight, stepTop, stepBot, N=4):
    Nu = N//2
    
    topLeft = range(topLeft, -Nu+topLeft-1, stepTop)
    topRight = range(topRight, -Nu+topRight-1, stepTop)
    botLeft = range(botLeft, botLeft + Nu*stepBot + 1, stepBot)
    botRight = range(botRight, botRight + Nu*stepBot + 1, stepBot)
    print(topLeft)
    print(topRight)
    print(botLeft)
    print(botRight)
    return [list(a) for a in zip(topLeft, topRight, botLeft, botRight)]
    
    

In [66]:
Acontr = genAContr(1, 3, 5, 4, N)

In [61]:
genUContr(-1, -2, Acontr[0][1], Acontr[1][1], -2, 2*(Acontr[1][1] - Acontr[0][1]))

[[-1, -2, 3, 7], [-3, -4, 11, 15]]

In [119]:
N = 6

In [120]:
# Acontr
AContr = genAContr(1, 4, 7, 6, N)
AContr

[[1, 4, 7],
 [7, 10, 13],
 [13, 16, 19],
 [19, 22, 25],
 [25, 28, 31],
 [31, 34, 37]]

In [121]:
# AdagContr
ADagContr = genAContr(2, 5, 8, 6, N)
ADagContr

[[2, 5, 8],
 [8, 11, 14],
 [14, 17, 20],
 [20, 23, 26],
 [26, 29, 32],
 [32, 35, 38]]

In [122]:
def genUContr(topLeft, topRight, botLeft, botRight, stepTop, stepBot, Nu=2):
    
    topLeft = range(topLeft, topLeft + stepTop*Nu, stepTop)
    topRight = range(topRight, topRight + stepTop*Nu, stepTop)
    botLeft = range(botLeft, botLeft + Nu*stepBot + 1, stepBot)
    botRight = range(botRight, botRight + Nu*stepBot + 1, stepBot)
    print(topLeft)
    print(topRight)
    print(botLeft)
    print(botRight)
    return [list(a) for a in zip(topLeft, topRight, botLeft, botRight)]
    
    

In [129]:
# Even layer U for A
evenAContr = genUContr(3, 9, 4, 10, 12, 12, 3)
#evenAContr[0][0] = -1
#evenAContr[-1][1] = -N
evenAContr

range(3, 39, 12)
range(9, 45, 12)
range(4, 41, 12)
range(10, 47, 12)


[[3, 9, 4, 10], [15, 21, 16, 22], [27, 33, 28, 34]]

In [130]:
# Odd layer U for A
oddAContr = genUContr(-1, -2, 9, 15, -2, 12, 2)
oddAContr

range(-1, -5, -2)
range(-2, -6, -2)
range(9, 34, 12)
range(15, 40, 12)


[[-1, -2, 9, 15], [-3, -4, 21, 27]]

In [131]:
# Even layer for Adag
evenADagContr = genUContr(6, 12, 5, 11, 12, 12, 3)
evenADagContr[0][0] = evenAContr[0][0]
evenADagContr[-1][1] = evenAContr[-1][1]
evenADagContr

range(6, 42, 12)
range(12, 48, 12)
range(5, 42, 12)
range(11, 48, 12)


[[3, 12, 5, 11], [18, 24, 17, 23], [30, 33, 29, 35]]

In [132]:
# Odd layer for Adag
oddADagContr = genUContr(-5, -6, 12, 18, -2, 12, 2)
oddADagContr

range(-5, -9, -2)
range(-6, -10, -2)
range(12, 37, 12)
range(18, 43, 12)


[[-5, -6, 12, 18], [-7, -8, 24, 30]]

In [133]:
# l/r contr
lcontr = [ADagContr[0][0], AContr[0][0]]
rcontr = [AContr[-1][2], ADagContr[-1][2]]

In [134]:
print(lcontr)
print(rcontr)
print(AContr)
print(ADagContr)
print(evenAContr)
print(oddAContr)
print(evenADagContr)
print(oddADagContr)

[2, 1]
[37, 38]
[[1, 4, 7], [7, 10, 13], [13, 16, 19], [19, 22, 25], [25, 28, 31], [31, 34, 37]]
[[2, 5, 8], [8, 11, 14], [14, 17, 20], [20, 23, 26], [26, 29, 32], [32, 35, 38]]
[[3, 9, 4, 10], [15, 21, 16, 22], [27, 33, 28, 34]]
[[-1, -2, 9, 15], [-3, -4, 21, 27]]
[[3, 12, 5, 11], [18, 24, 17, 23], [30, 33, 29, 35]]
[[-5, -6, 12, 18], [-7, -8, 24, 30]]
